In [ ]:
import pandas as pd
import ast
import tokenize
from io import StringIO
import textwrap
import re
import autopep8
import time
from tqdm import tqdm

# --- Patch legacy Python 2 exceptions ---
def fix_legacy_exception_syntax(code: str) -> str:
    return re.sub(r'except\s+(\w+)\s*,\s*(\w+):', r'except \1 as \2:', code)

# --- Fallback: Regex-based docstring remover ---
def remove_docstrings_fallback(code: str) -> str:
    return re.sub(r'(""".*?"""|\'\'\'.*?\'\'\')', '', code, flags=re.DOTALL)

# --- Robust AST-based docstring remover with fallback ---
def remove_docstrings(source_code: str) -> str:
    source_code = fix_legacy_exception_syntax(source_code)
    lines = source_code.splitlines()
    try:
        tree = ast.parse(source_code)
    except SyntaxError:
        return remove_docstrings_fallback(source_code)

    for node in ast.walk(tree):
        if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef, ast.ClassDef, ast.Module)):
            if not node.body:
                continue
            first_stmt = node.body[0]
            if (
                isinstance(first_stmt, ast.Expr)
                and isinstance(first_stmt.value, ast.Constant)
                and isinstance(first_stmt.value.value, str)
            ):
                start = first_stmt.lineno - 1
                end = getattr(first_stmt, 'end_lineno', start + len(first_stmt.value.value.splitlines()))
                for i in range(start, end):
                    if 0 <= i < len(lines):
                        lines[i] = ''
    return '\n'.join(lines)

# --- Remove all `#` comments with robust indentation fixing ---
def remove_all_comments(code: str) -> (str, bool):

    attempts = 0
    max_attempts = 1
    failed = False
    while attempts < max_attempts:
        try:
            io_obj = StringIO(code)
            tokens = list(tokenize.generate_tokens(io_obj.readline))
            break  # Successful tokenization, exit loop.
        except IndentationError as e:
            attempts += 1
            code = autopep8.fix_code(code)
    else:
        # If we exceed max_attempts, flag the issue and return code unchanged.
        failed = True
        return code, failed

    # Build code output from tokens, skipping those with type COMMENT.
    output = ""
    last_lineno = -1
    last_col = 0
    for tok in tokens:
        token_type, token_string, start, end, _ = tok
        if token_type == tokenize.COMMENT:
            continue
        sline, scol = start
        eline, ecol = end
        if sline > last_lineno:
            output += "\n" * (sline - last_lineno - 1)
            last_col = 0
        if scol > last_col:
            output += " " * (scol - last_col)
        output += token_string
        last_lineno = eline
        last_col = ecol
    return output, failed

# --- Remove extra blank lines ---
def remove_extra_blank_lines(code: str) -> str:
    cleaned_lines = []
    previous_blank = False
    for line in code.splitlines():
        if line.strip() == "":
            if not previous_blank:
                cleaned_lines.append("")
                previous_blank = True
        else:
            cleaned_lines.append(line.rstrip())
            previous_blank = False
    return "\n".join(cleaned_lines)

# --- Fix indentation using autopep8 ---
def fix_indentation(code: str) -> str:
    return autopep8.fix_code(code)

# --- Full pipeline ---
def split_code_and_comments(source_code: str) -> (str, bool):
    # First, fix indentation issues and dedent.
    source_code = fix_indentation(source_code)
    source_code = textwrap.dedent(source_code)
    # Remove docstrings.
    no_docstrings = remove_docstrings(source_code)
    # Remove inline comments and get flag.
    no_comments, flag = remove_all_comments(no_docstrings)
    # Remove extra blank lines.
    final_code = remove_extra_blank_lines(no_comments)
    return final_code, flag

def clean_code_column(input_csv: str, code_column: str = "code"):
    df = pd.read_csv(input_csv)
    cleaned_codes = []
    flags = []
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        final_code, flag = split_code_and_comments(str(row[code_column]))
        cleaned_codes.append(final_code)
        flags.append(flag)
    df["cleaned_code"] = cleaned_codes
    # Record indices where remove_all_comments returned code unchanged (flag True)
    df["remove_all_comments_issue"] = flags
    return df

# --- Step 6: Run it ---
if __name__ == "__main__":
    input_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/raw_data/cosqa_queries_code_corpus.csv"
    start = time.time()
    df = clean_code_column(input_path)
    output_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/pre_processing_COSQA/COSQA_cleaned_code.csv"
    df.to_csv(output_path, index=False)
    end = time.time()
    print(f"Cleaned CSV written to: {output_path} in {end-start} seconds")


 62%|██████▏   | 12832/20604 [01:28<00:51, 151.49it/s]<string>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
100%|██████████| 20604/20604 [02:22<00:00, 144.82it/s]


Cleaned CSV written to: /work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/pre_processing_COSQA/COSQA_cleaned_code.csv in 142.67305779457092 seconds
